# Field Integration Techniques

Many analysis techniques for vector fields require solving an initial value problem for an arbitrary set of seed points and evaluating such solutions at a chosen resolution. Kamodo makes it easy to generate fieldline solutions by providing a function decorator that wraps scipy's powerful `solve_ivp` function. Each family of solutions is represented by a single function of a complex parameter. We illustrate the flexibility of this approach in the example below.

In [2]:
# initialize
from plotly.offline import iplot, plot, init_notebook_mode
# init_notebook_mode(connected = True)

from kamodo import Kamodo, event, pointlike, kamodofy, solve
import numpy as np
import pandas as pd

## Dipole field model

We use the following dipole field model that can accept (m,) and (1,m), and (n,m) arrays.

In [3]:
def Bdip(rvec):
    """Need math to work in a variety of arg shapes"""
    muvec = Bdip.muvec    
    r = np.linalg.norm(rvec, axis = 1)
    r[r==0] = np.nan

    try:
        rhat = rvec/r
    except:
        rhat = (rvec.T/r).T

    try:
        result = 3*np.dot(rhat, muvec.T)
    except:
        result = 3*np.dot(rhat.T, muvec.T).T


    result = (rhat.T*result).T

    try:
        result = result - muvec
    except:
        result = (result - muvec.T).T

    try:
        result = result/r**3
    except:
        result = (result.T/r**3).T

    return result

# set dipole moment
Bdip.muvec = np.array([0, 0, -1]) 

# pointlike enforces dimensionality
Bdip = pointlike(Bdip, '(n,m)->(n,m)', [np.float], squeeze = 0)

In [29]:
kamodo = Kamodo()

In [30]:
kamodo['Bvec'] = Bdip # register the dipole field
kamodo

Kamodo([(Bvec(rvec), <function __main__.Bdip(rvec)>),
        (Bvec, <function __main__.Bdip(rvec)>)])

## Normalization

Instead of solving the initial value problem on the original field, we will be solving on the normalized field. This will mean that the integral path is the same as the arclength, allowing us to control the visual fidelity of the resulting field.

Create a normalization function to be applied to our field

In [31]:
@kamodofy(equation = "$$\\hat{n}(\\vec{y}) = \\vec{y}/\\sqrt{\\vec{y} \\cdot \\vec{y}} $$")
@pointlike(signature = '(m,n)->(m,n)', squeeze = 0)
def normalized(yvec):   
    r = np.linalg.norm(yvec, axis = 1)
    r[r==0] = np.nan

    try:
        return yvec/r
    except:
        return (yvec.T/r).T


kamodo['nhat'] = normalized

Create a normalized field

In [32]:
kamodo['bhat'] = "nhat(Bvec)"
kamodo

Kamodo([(Bvec(rvec), <function __main__.Bdip(rvec)>),
        (Bvec, <function __main__.Bdip(rvec)>),
        (nhat(yvec), <function __main__.normalized(yvec)>),
        (nhat, <function __main__.normalized(yvec)>),
        (bhat(rvec), <function _lambdifygenerated(rvec)>),
        (bhat, <function _lambdifygenerated(rvec)>)])

## Solving the initial value problem

Generate a set of seed points for integration

In [33]:
x0 = np.linspace(-np.pi,np.pi,6)
y0 = np.linspace(-np.pi,np.pi,6)
z0 = 1

seeds = np.array(np.column_stack([c.ravel() for c in np.meshgrid(x0,y0,z0)]))

Create a stopping boundary for field line integrator

In [34]:
@event
def boundary(s, rvec):
    r = np.linalg.norm(rvec)
    
    if np.isnan(r):
        result = 0
    else:
        result = r - 1
    return result


Solve the initial value problem for the normalized field

In [35]:
kamodo['svec'] = solve(kamodo.bhat, # the field to be solved
                       seeds, # the initial positions
                       's', # the name of the integration parameter
                       (0,30), # the span to integrate over
                       npoints = 60, # the number of points to evaluate the solution
                       events = boundary, # stop at the boundary
                      )
kamodo

Kamodo([(Bvec(rvec), <function __main__.Bdip(rvec)>),
        (Bvec, <function __main__.Bdip(rvec)>),
        (nhat(yvec), <function __main__.normalized(yvec)>),
        (nhat, <function __main__.normalized(yvec)>),
        (bhat(rvec), <function _lambdifygenerated(rvec)>),
        (bhat, <function _lambdifygenerated(rvec)>),
        (svec(s),
         <function kamodo.util.solve.<locals>.decorator_solve.<locals>.solution(s=array([ 0.-6.61016949j,  0.-6.10169492j,  0.-5.59322034j,  0.-5.08474576j,
        0.-4.57627119j,  0.-4.06779661j,  0.-3.55932203j,  0.-3.05084746j,
        0.-2.54237288j,  0.-2.03389831j,  0.-1.52542373j,  0.-1.01694915j,
        0.-0.50847458j,  0.+0.j        ,  0.+0.50847458j,  0.+1.01694915j,
        0.+1.52542373j,  0.+2.03389831j,  0.+2.54237288j,  0.+3.05084746j,
        0.+3.55932203j,  0.+4.06779661j,  1.-5.59322034j,  1.-5.08474576j,
        1.-4.57627119j,  1.-4.06779661j,  1.-3.55932203j,  1.-3.05084746j,
        1.-2.54237288j,  1.-2.03389831j,  1.-1.

The solver returns a family of solutions, represented as a single function of a complex array, $\vec{s}(s)$ where
$s$ is a complex array. 

## Evaluating the Solutions
On evaluation, $\vec{s}(s)$ returns a pandas dataframe.

In [36]:
kamodo.svec().head()

0         1         2
seed integral                               
0.0  -6.610169 -0.347554 -0.347554 -0.924068
     -6.101695 -0.615865 -0.615865 -1.261579
     -5.593220 -0.922720 -0.922720 -1.525495
     -5.084746 -1.256963 -1.256963 -1.713145
     -4.576271 -1.608411 -1.608411 -1.822192

When using the default argument above, the solution evaluates at a resolution of npoints/span, stopping at the boundary.

## Complex parameterization
Kamodo represents the family of solutions to the initial value problem as a single function of a complex array.

The floor of the real part of the input parameter corresponds to the original seed array:

In [12]:
kamodo.svec([0,1,2]).values

array([[-3.14159265, -3.14159265,  1.        ],
       [-1.88495559, -3.14159265,  1.        ],
       [-0.62831853, -3.14159265,  1.        ]])

compare with original seeds:

In [13]:
seeds[[0,1,2]]

array([[-3.14159265, -3.14159265,  1.        ],
       [-1.88495559, -3.14159265,  1.        ],
       [-0.62831853, -3.14159265,  1.        ]])

The imaginary part denotes the integral along the corresponding solution. Here, we can choose evaluation points that were not in the original solution. Parameters outside the original span will be extrapolated.

In [27]:
kamodo.svec([-6j, -5j, 0, 5j, 6j, 4 + 4j, 4 -5.777j])

0         1         2
seed integral                              
0.0  -6.000   -0.674467 -0.674467 -1.320541
     -5.000   -1.314574 -1.314574 -1.737228
      0.000   -3.141593 -3.141593  1.000000
      5.000   -0.120683 -0.120683  0.491788
      6.000    0.115024  0.115024 -0.404619
4.0   4.000    0.094188 -0.156979  0.481816
     -5.777    0.234822 -0.391371 -0.826931

## Plotting Fieldlines

We can quickly generate plots for all fieldlines at the default resolution by calling plot with the name of the fieldlines solution.

In [15]:
import plotly.io as pio

In [16]:
fig = kamodo.plot('svec')
pio.write_image(fig, 'images/fieldlines.svg')

![images/fieldlines.svg](images/fieldlines.svg)

To show the direction of the field at each point, we can evaluate $\hat{B}(\vec{s}(s))$

In [17]:
fig = kamodo.plot('svec', 
                  Bhat = dict(rvec = kamodo.svec()))
pio.write_image(fig,'images/fieldlines_vectors.svg')

![fieldlines](images/fieldlines_vectors.svg)

## Integration totals

To compute the total integral for each fieldline individually, we need a function to subtract the integration results at the endpoints.

In [47]:
def integral(fieldline):
    endpoints = fieldline.reset_index().integral.iloc[[0,-1]]
    return endpoints.values[-1] - endpoints.values[0]

In [74]:
totals = []
for seed, fieldline in kamodo.svec().groupby(level = 'seed'):
    totals.append(integral(fieldline))
    
totals[:5]

[10.677966101694915,
 8.64406779661017,
 7.627118644067796,
 7.627118644067796,
 8.64406779661017]

Alternatively, we can use pandas' aggregation methods to apply our function on each fieldline.

In [ ]:
pd.DataFrame.groupby.a

In [79]:
kamodo.svec().groupby(level='seed').aggregate(integral)

,0,1,2
seed,,,
0.0,10.677966,10.677966,10.677966
1.0,8.644068,8.644068,8.644068
2.0,7.627119,7.627119,7.627119
3.0,7.627119,7.627119,7.627119
4.0,8.644068,8.644068,8.644068
5.0,10.677966,10.677966,10.677966
6.0,8.644068,8.644068,8.644068
7.0,6.610169,6.610169,6.610169
8.0,5.084746,5.084746,5.084746
